# Grab AI for SEA Challenge 1 (Traffic Management)
This is a Time series problem. <br>This challenge is done in Python by method <b>Seasonal AutoRegressive Integrated Moving Average (SARIMA)</b> method.
##### Name: Liam Liew

## Processing and reading required Data 

I use numpy and pandas to clean the data from training.csv file.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pmdarima
import statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
%matplotlib inline

In [ ]:
full_data = pd.read_csv("training.csv")
full_data.count()

##### Recalculate all the timestamp to 'time_order' attribute according to day and the timestamp. 

Day 1 00:00 = 1, <br> 
Day 1 00:15 = 2, <br> 
Day 1 01:00 = 4, <br> 
Day 2 00:00 = 97,<br> 
Day 2 13:15 = 150<br> 

In [ ]:
# split the timestamp accordingly and calculate time_order
full_data["time_order"] = ((full_data["timestamp"].str.split(":", n=1, expand=True)[0].astype(int) * 60 + \
                  full_data["timestamp"].str.split(":", n=1, expand=True)[1].astype(int) + \
                 (full_data["day"]-1)*24*60)/15).astype(int)


In [ ]:
# remove the unnecessary columns
full_data = full_data.drop(["day","timestamp"], axis = 1)
full_data.head()

In [ ]:
# add another column 'count' to calculate how many data does each geohash6 location has in the training file
full_data_indexed = full_data.set_index(["geohash6"])
full_data_indexed['count'] = full_data_indexed.groupby('geohash6').count().iloc[:,0]

# filter out the geohash6 that lack of data
full_data_indexed = full_data_indexed[full_data_indexed['count']>=2000]

In [ ]:
# sort the data and separate it in a list with different location
sorted_full_data_indexed = full_data_indexed.sort_values(["count","geohash6","time_order"], ascending = False)
separated = [sorted_full_data_indexed.loc[geohash6,:] for geohash6 in sorted_full_data_indexed.index.unique()]

# this list is for getting test data from dictionary purpose
geohash6 = sorted_full_data_indexed.index.unique()

In [ ]:
# create an array to make sure that NULL value can be filled in the next few steps
time_order_list = pd.DataFrame({'time_order' : np.arange(5856)})

In [ ]:
# for every location in the separated_data, merge the data with time_oreder_list so that we can fill the NULL value for some data
# sort the time_order and delete the 'count' column after that 
separated = [pd.merge(separated[i], time_order_list, how='right', on='time_order').sort_values('time_order').drop(["count"], axis = 1).set_index("time_order") for i in range(len(separated))]

In [ ]:
separated_data = {}
for i,geo in enumerate(geohash6):
    separated_data[geo] = separated[i]

In [ ]:
# use Adfuller to test if the data is stationary for this time series problem
from statsmodels.tsa.stattools import adfuller
def test_stationarity(data_set):
    """
    @parameter 
    data_set: data for a location including time_order and demand
    
    @return
    this function will print the output for adfuller test and also the stationarity graph
    """
    
    #Determing rolling statistics
    rolmean = data_set["demand"].rolling(window=96).mean()
    rolstd = data_set["demand"].rolling(window=96).std()

    #Plot rolling statistics:
    orig = plt.plot(data_set, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(data_set["demand"], autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

### Identification

In this challenge, we need to forecast 5 results of traffic demand in a particular location. Provided 14 consecutive days from the dataset, ending at timestamp T and predict T+1 to T+5, we need to check how many days do we need in advance to forecast the 5 results.

In this challenge, each day contains 96 different demand. <br/>Since we only need for 5 time intervals prediction with 14 days dataset, I separate out by different day to predict to increase the accuracy, meaning to say I will only use <b>previous N days</b> to predict based on the result we get.(Explantaion in word document)

#### **Use the correct function for testing (different from training)

In [ ]:
plt.rcParams['figure.figsize'] = [20, 5]

def grabData(data_set, start, testday):
    """
    @parameter 
    data_set: data for a location including time_order and demand
    start: random 'time_order' number that to start for the tranning set 
    testday: the day that used to predict the next 5 time interval
    
    @return
    this will return a stationarity data that to put in SARIMA model
    """
    test = data_set[start: start + testday*96]

    test = test.reset_index(drop = True)

    # fill the gap if there is any missing or NULL value and if first few values are empty then back fill
    final_test = test.interpolate().fillna(method = 'bfill')
    return final_test
    
def grabDataTesting(data_set, testday):
    """
    @parameter 
    data_set: data for a location including time_order and demand
    testday: the day that used to predict the next 5 time interval
    
    @return
    this will return a stationarity data that to put in SARIMA model
    """
    test = data_set[-96*testday:]

    test = test.reset_index(drop = True)

    # fill the gap if there is any missing or NULL value and if first few values are empty then back fill
    final_test = test.interpolate().fillna(method = 'bfill')
    return final_test

def stationarity(final_test):
    """
    @parameter 
    final_test: the data used to test and forecast the demand from T+1 to T+5 interval
    
    @return
    this will return a stationarity data that to plot ACF and PACF Charts
    """

    # two differencing here to make the seasonal traffic data become a stationary data before going for SARIMA model
    # d = 1, D = 1 for SARIMA model
    diff1 = final_test - final_test.shift()
    stationarity = diff1 - diff1.shift(96)
    plt.plot(stationarity)
    stationarity = stationarity.dropna()
    test_stationarity(stationarity)
    return stationarity

# 100 is the random number to start and 4 is the 4 continuos day after time_order 100, it is used to predict after that
# comment out this when doing testing
final_test = grabData(separated_data[geohash6[3]],100,4)
stationarity = stationarity(final_test)

# uncomment it when doing testing
# final_test = grabDataTesting(separated_data["geohash6--Name"])
# stationarity = stationarity(final_test)

## Observation
As in this step, I will observe whether there's a spike down from one bar to another in both charts. <br><b>Auto Correlation</b> is to determine <b>q</b> and <b>Q</b> value. <br> <b>Partial Auto Correlation</b> is for <b>p</b> and <b>P</b> value. We use the number 96 below is because there are 96 data in a day and it is seasonal period.
#### To observe the Auto correlation and Partial auto correlation graph for the SARIMA(p,d,q)(P,D,Q)96. 
This is usually will be a range or number and waiting to be tested out by SARIMAX model according to AIC number. <br> In this case, we will use python to roughly tell us what is the range to use for p,q, P and Q.

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.rcParams['figure.figsize'] = [20, 20]

from statsmodels.tsa.stattools import acf, pacf

def acfPacf(stationarity):
    """
    @parameter 
    stationarity: stationarity data for a location including time_order and demand
    
    @return
    it will plot two chars: ACF and PACF
    """    
    lag_acf = acf(stationarity, nlags=100)
    lag_pacf = pacf(stationarity, nlags=100, method='ols')

    plt.subplot(211)
    plt.bar(np.arange(len(lag_acf)),lag_acf)
    plt.axhline(y=0,linestyle = "--", color = 'gray')
    plt.axhline(y = -1.96/np.sqrt(len(stationarity)), linestyle = "--", color = 'gray')
    plt.axhline(y = 1.96/np.sqrt(len(stationarity)), linestyle = "--", color = 'gray')
    plt.title("Auto Correlation Bar Chart")

    plt.subplot(212)
    plt.bar(np.arange(len(lag_pacf)),lag_pacf)
    plt.axhline(y=0,linestyle = "--", color = 'gray')
    plt.axhline(y = -1.96/np.sqrt(len(stationarity)), linestyle = "--", color = 'gray')
    plt.axhline(y = 1.96/np.sqrt(len(stationarity)), linestyle = "--", color = 'gray')
    plt.title("Partial Auto Correlation Bar Chart")
    
acfPacf(stationarity)

In my testing case, it is easy to determine that <b>p</b> is somehow in <b>range[0,1,2]</b>, <b>P</b>, <b>q</b> and <b>Q</b>  is either <b>0</b> or <b>1</b>.

# Skip this for testing
we found out SARIMAX (1,1,1)(1,1,0)96 OR SARIMAX (1,1,0)(1,1,0)96 is the one with lowest AIC, which is the best model to be used in this challenge

In [ ]:
model_list = []
model_fit_list= []

# this is for training purpose to test out which is the best model for SARIMA
# in the testing we found out SARIMAX (1,1,1)(1,1,0)96 OR SARIMAX (1,1,0)(1,1,0)96
for day in range(3,8):
    print(day)
    print("RESULTS:")
    p_list = [0,1,2]
    P_list = q_list = Q_list = [0,1]

    final_test = grabData(separated_data[geohash[3]],100,day)
    
    # 96 is a seasonal period which there are 96 data in a day
    final_model = SARIMAX(final_test, order=(0,1,0), seasonal_order=(0,1,0,96))
    final_model_fit = final_model.fit()
    for p in p_list:
        for P in P_list:
            for q in q_list:
                for Q in Q_list:
                    if (p!=0 or q!=0 or P!=0 or Q!=0):
                        model = SARIMAX(final_test, order=(p,1,q), seasonal_order=(P,1,Q,96))
                        try:
                            model_fit = model.fit()
                            if final_model_fit.aic > model_fit.aic:
                                final_model = model
                            print( "SARIMAX model: non-seasonal(",p,", 1, ",q,") seasonal(",P,", 1, ",Q,") AIC value: ", model_fit.aic)
                        except:
                            print( "SARIMAX model: non-seasonal(",p,", 1, ",q,") seasonal(",P,", 1, ",Q,") AIC value: NAN")


    # model_list and model_fit_list will add the best model (lowest AIC value) for different number of day as input
    model_list.append(final_model.fit())
    model_fit_list.append(final_model_fit.forecast(5))
    

# Use this for testing
For the previous step, we found out SARIMAX (1,1,1)(1,1,0)96 OR SARIMAX (1,1,0)(1,1,0)96. Thus we compare only this two model with the lowest AIC.

In [ ]:
# change geohash[3] to a geohash6 name and second parameter for previous consecutive day for testing 
# comment out this for testing
final_test = grabData(separated_data[geohash6[3]],100,4)

# ---------------------------uncomment this for testing-------------------
# second parameter is suggested to be 4 continuos day to predict the next 5 interval
# final_test = grabDataTesting(separated_data[geohash6[3]],4)

# 96 is a seasonal period which there are 96 data in a day
final_model1 = SARIMAX(final_test, order=(1,1,1), seasonal_order=(1,1,0,96))
final_model2 = SARIMAX(final_test, order=(1,1,0), seasonal_order=(1,1,0,96))
fit1 = fit2 = True

try:
    final_model_fit1 = final_model1.fit()
except:
    fit1 = False
    print("SARIMA (1,1,1)(1,1,0,96) is NA, not enough data. (Add the number of the day to test)")

try:
    final_model_fit2 = final_model2.fit()
except:
    fir2 = False
    print("SARIMA (1,1,0)(1,1,0,96) is NA, not enough data. (Add the number of the day to test)")

    
if fit1 and fit2:

    if final_model_fit1.aic > final_model_fit2.aic:
        final_model = final_model2
        final_model_fit = final_model.fit()
    else:
        final_model = final_model1
        final_model_fit = final_model.fit()

elif fit1 and not fit2:
    final_model = final_model1
    final_model_fit = final_model.fit()

else:
    final_model = final_model2
    final_model_fit = final_model.fit()
        
        
final_demand = final_model_fit.forecast(5)

### Plot the Prediction and the actual data

In [ ]:
plt.plot(final_demand)
# comment out this for testing
plt.plot(grabData(separated_data[geohash6[3]],100,5)[:-91])

# uncomment this for testing
# plt.plot(grabDataTesting(separated_data[geohash6[3]],4))

In [ ]:
final_model_fit.plot_diagnostics(figsize=(15, 12))

## Calculate the RMSE based on the actual and predicted value

In [ ]:
# comment out this for testing
RMSE = np.sum((final_demand.as_matrix() - grabData(separated_data[geohash6[3]],100,5)[-96:-91].values.reshape(-1))**2)/len(final_demand)

# uncomment this for testing
# RMSE = np.sum((final_demand.as_matrix() - grabDataTesting(separated_data[geohash6[3]],4)[-96:-91].values.reshape(-1))**2)/len(final_demand)

RMSE